# __SPS Commerce Installation Tutorial__

## __1. Get an invite on the account of the client__ 

Send the client this link : https://help.analytics.spscommerce.com/en/articles/7170512-how-to-add-a-new-user
Make sure to ask them to give Administrator Access as well as an access to all products / retailer.

## __2. Create a Web Service Application__

Go into the Dev Center of the platform and create a Web Service Application.
Retrieve the __CLIENT_ID__ and the __CLIENT_SECRET__.

In [ ]:
import yaml
import os 
import requests

spsCredentials = open("../server/src/var/login_credentials/sps_login.yml", "r")
spsCredentials = yaml.load(spsCredentials, Loader=yaml.FullLoader)

CLIENT_ID = spsCredentials['sps_commerce']['CLIENT_ID']
CLIENT_SECRET = spsCredentials['sps_commerce']['CLIENT_SECRET']
print(CLIENT_ID, CLIENT_SECRET)

### __3. Allow app access__

Fill the params and click the printed URL.
Get the code.

In [ ]:
params = {'audience':'api://api.spscommerce.com/',
          'client_id':CLIENT_ID,
          'redirect_uri':'https://www.bravosierra.com/account/login',
          'response_type':'code',
          'scope':'offline_access'
          }
url = f"https://auth.spscommerce.com/authorize?redirect_uri={params['redirect_uri']}&response_type={params['response_type']}&client_id={params['client_id']}&scope={params['scope']}&audience={params['audience']}"
print(url)

In [ ]:
code = 'xxx'

In [ ]:
url = f"https://auth.spscommerce.com/oauth/token"
params = {'client_id':CLIENT_ID,
            'client_secret':CLIENT_SECRET,
            'code':code,
            'grant_type':'authorization_code',
            'redirect_uri':'https://www.bravosierra.com/account/login'
            }
response = requests.post(url, params)
print(response.json())
ACCESS_TOKEN = response.json()['access_token']
REFRESH_TOKEN = response.json()['refresh_token']

In [ ]:
spsCredentials['sps_commerce']['ACCESS_TOKEN'] = ACCESS_TOKEN
spsCredentials['sps_commerce']['REFRESH_TOKEN'] = REFRESH_TOKEN

with open("../server/src/var/login_credentials/sps_login.yml", "w") as file:
    yaml.dump(spsCredentials, file)

In [ ]:
url = f"https://api.spscommerce.com/auth-check"

headers = {'Authorization':"Bearer " + ACCESS_TOKEN,}
response = requests.post(url, headers=headers)
print(response.status_code)

In [ ]:
def refresh_token(token,refresh_token):
    url = f"https://api.spscommerce.com/auth-check"
    headers = {'Authorization':"Bearer " + token}
    response = requests.post(url, headers=headers)

    if response.status_code == 204:
        return token, refresh_token
    else:
        url = f"https://auth.spscommerce.com/oauth/token"
        params = {'client_id':CLIENT_ID,
                'client_secret':CLIENT_SECRET,
                'refresh_token':refresh_token,
                'grant_type':'refresh_token',
                }
        response = requests.post(url, params)
        ACCESS_TOKEN = response.json()['access_token']
        REFRESH_TOKEN = response.json()['refresh_token']
        return ACCESS_TOKEN, REFRESH_TOKEN